<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Copenhagen_Denmark_meteostat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
pip install meteostat

In [46]:
from meteostat import Stations, Hourly
from datetime import datetime
import pandas as pd
import warnings

# Suppress warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# --- CONFIGURATION ---
# Copenhagen City Center Coordinates
LAT = 55.6761
LON = 12.5683
START = datetime(2024, 1, 1)
END = datetime(2025, 1, 1)

print(f"Searching for stations near {LAT}, {LON}...")

# 1. SEARCH for Nearby Stations
stations = Stations()
stations = stations.nearby(LAT, LON)
# Look for stations with data in 2024 (inventory checks)
stations = stations.inventory('hourly', (START, END))
# Get the top 3 closest matches
nearby_stations = stations.fetch(3)

print("\n--- Nearby Stations Found ---")
print(nearby_stations[['name', 'country', 'hourly_start', 'hourly_end']])
print("-----------------------------")

# 2. SELECT the Best Station
# We prefer the one with a valid WMO ID (usually airports/major stations)
# or simply the first one that works.
if not nearby_stations.empty:
    # Pick the first one in the list (usually closest)
    # OR pick the one you know works, e.g., 'Kastrup' if visible.
    # Here we just take the first result returned by the inventory filter.
    best_station_id = nearby_stations.index[0]
    station_name = nearby_stations.iloc[0]['name']

    print(f"\n> Selected Station: {station_name} (ID: {best_station_id})")
    print(f"> Fetching Data...")

    # 3. FETCH Data
    data = Hourly(best_station_id, START, END)
    data = data.fetch()

    if not data.empty:
        df = data.reset_index()

        # Rename & Clean
        rename_map = {
            'time': 'Time', 'temp': 'Temperature_C',
            'wspd': 'Wind_Speed_kmh', 'rad': 'GHI_Wm2'
        }
        df = df.rename(columns=rename_map)

        if 'Wind_Speed_kmh' in df.columns:
            df['Wind_Speed_ms'] = df['Wind_Speed_kmh'] / 3.6

        df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
        df['Hour'] = df['Time'].dt.strftime('%H:%M')

        # Formatting Output
        cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms']
        if 'GHI_Wm2' in df.columns: cols.append('GHI_Wm2')

        df_final = df[[c for c in cols if c in df.columns]]

        print("-" * 30)
        print(df_final.head())
        print("-" * 30)

        filename = f"Copenhagen_{best_station_id}_2024.csv"
        df_final.to_csv(filename, index=False)
        print(f"Success! Saved to {filename}")

    else:
        print("Selected station returned no data.")
else:
    print("No stations found nearby with data inventory for 2024.")

Searching for stations near 55.6761, 12.5683...

--- Nearby Stations Found ---
                        name country hourly_start hourly_end
id                                                          
06180   Koebenhavn / Kastrup      DK   1929-08-01 2025-12-15
06170  Koebenhavn / Roskilde      DK   1977-04-19 2025-12-15
02635                  Malmo      SE   2018-01-27 2025-12-15
-----------------------------

> Selected Station: Koebenhavn / Kastrup (ID: 06180)
> Fetching Data...
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms
0  2024-01-01  00:00            5.4       4.111111
1  2024-01-01  01:00            5.4       5.111111
2  2024-01-01  02:00            5.6       5.694444
3  2024-01-01  03:00            5.5       5.111111
4  2024-01-01  04:00            5.6       4.111111
------------------------------
Success! Saved to Copenhagen_06180_2024.csv
